In [1]:
import requests
 
print(requests.get("http://localhost:8080/v1/models/cc-model").json())

{'model_version_status': [{'version': '1696351102', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}]}


In [2]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [3]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [16]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint2 = "http://localhost:8080/v1/models/cc-model:predict"
    response = requests.post(endpoint2, data=json_data)

    prediction = response.json()["predictions"][0][0]

    print(prediction)

    if prediction < 0.3:
        return "Normal"
    else:
        return "Stroke"

In [17]:
df = pd.read_csv("data/healthcare-dataset-stroke-data-cleaned.csv")

columns = df.columns.values
print(columns)

rand = random.randint(0, len(columns))

features = df[df['stroke'] == 0].values[rand]
inputs = {key: value for key, value in zip(columns, features)}
actual_label_key = 'Stroke' if inputs['stroke'] == 1 else 'Normal'
print(inputs)

print("Predicted: {0}, Actual: {1}".format(make_predictions(inputs), actual_label_key))

['gender' 'age' 'hypertension' 'heart_disease' 'ever_married' 'work_type'
 'Residence_type' 'avg_glucose_level' 'bmi' 'smoking_status' 'stroke']
{'gender': 'Female', 'age': 70.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Rural', 'avg_glucose_level': 69.04, 'bmi': 35.9, 'smoking_status': 'formerly smoked', 'stroke': 0}
0.464222074
Predicted: Stroke, Actual: Normal


In [24]:
test_round = 1000
correct_test = 0
for _ in range(1000):
    rand = random.randint(0, len(columns))

    features = df.values[rand]
    inputs = {key: value for key, value in zip(columns, features)}
    print(inputs)
    actual_label_key = 'Stroke' if inputs['stroke'] == 1 else 'Normal'

    if actual_label_key == make_predictions(inputs):
        correct_test += 1

    print("Predicted: {0}, Actual: {1}".format(make_predictions(inputs), actual_label_key))

print("Accuracy: {0}%".format(correct_test/test_round*100))

{'gender': 'Male', 'age': 67.0, 'hypertension': 0, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 228.69, 'bmi': 36.6, 'smoking_status': 'formerly smoked', 'stroke': 1}
0.499522805
0.499522805
Predicted: Stroke, Actual: Stroke
{'gender': 'Male', 'age': 67.0, 'hypertension': 0, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 228.69, 'bmi': 36.6, 'smoking_status': 'formerly smoked', 'stroke': 1}
0.499522805
0.499522805
Predicted: Stroke, Actual: Stroke
{'gender': 'Female', 'age': 49.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 171.23, 'bmi': 34.4, 'smoking_status': 'smokes', 'stroke': 1}
0.457711875
0.457711875
Predicted: Stroke, Actual: Stroke
{'gender': 'Female', 'age': 49.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Private', 'R